In [ ]:
!pip install -U spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy
import os
import numpy as np

In [ ]:
nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])

In [ ]:
data_path = 'training_data.csv'
features_path = 'training_features.csv'

In [ ]:
df_data = pd.read_csv(data_path)
df_features = pd.read_csv(features_path)

print(f"Loaded Raw Data: {df_data.shape}")
print(f"Loaded Features: {df_features.shape}")

Loaded Raw Data: (1138, 2)
Loaded Features: (1138, 20)


In [ ]:
vocab_levels = ['a1', 'a2', 'b1', 'b2', 'c1']
vocab_sets = {}
seen_words = set()

print("\nLoading Vocabulary Lists (Disjoint Mode)...")
for level in vocab_levels:
    filename = f"new_vocab_{level}.txt"
    try:
        with open(filename, 'r', encoding='utf-8') as f:

            raw_words = {line.strip().lower() for line in f if line.strip()}


            unique_words = raw_words - seen_words


            seen_words.update(raw_words)

            vocab_sets[level] = unique_words

        print(f"  - {level.upper()}: {len(vocab_sets[level])} unique words (Raw: {len(raw_words)})")
    except FileNotFoundError:
        print(f"  - ERROR: Could not find {filename}")
        vocab_sets[level] = set()




Loading Vocabulary Lists (Disjoint Mode)...
  - A1: 1006 unique words (Raw: 1006)
  - A2: 574 unique words (Raw: 1560)
  - B1: 928 unique words (Raw: 2482)
  - B2: 3208 unique words (Raw: 5500)
  - C1: 6467 unique words (Raw: 11956)


In [ ]:
def get_lemmas(text):
    if not isinstance(text, str):
        return []

    doc = nlp(text)


    lemmas = [token.lemma_.lower() for token in doc if token.is_alpha]
    return lemmas

In [ ]:
def get_vocab_ratios(lemma_list):
    if not lemma_list:
        return pd.Series([0.0] * 5, index=[f'ratio_{l}' for l in vocab_levels])

    total_tokens = len(lemma_list)
    ratios = {}

    for level in vocab_levels:
        count = sum(1 for lemma in lemma_list if lemma in vocab_sets[level])
        ratios[f'ratio_{level}'] = count / total_tokens

    return pd.Series(ratios)

In [ ]:
df_data['lemmas'] = df_data['text'].apply(get_lemmas)

In [ ]:
vocab_features = df_data['lemmas'].apply(get_vocab_ratios)

In [ ]:
df_extended = pd.concat([df_features, vocab_features], axis=1)

In [ ]:
cols = [c for c in df_extended.columns if c != 'label'] + ['label']
df_extended = df_extended[cols]

In [ ]:
output_filename = 'training_features_extended.csv'
df_extended.to_csv(output_filename, index=False)

In [ ]:
print(vocab_features.head())

   ratio_a1  ratio_a2  ratio_b1  ratio_b2  ratio_c1
0  0.575000  0.125000  0.033333  0.025000  0.033333
1  1.000000  0.000000  0.000000  0.000000  0.000000
2  0.600000  0.000000  0.000000  0.000000  0.000000
3  0.670213  0.053191  0.021277  0.010638  0.021277
4  0.636872  0.083799  0.044693  0.011173  0.022346


In [ ]:
print(df_data['lemmas'].head())

0    [сообщение, об, это, слово, мы, думать, каждый...
1                                       [я, не, знать]
2                          [у, меня, не, быть, словай]
3    [следуюший, пожалуйста, если, мне, не, нравить...
4    [мне, казаться, что, личный, телефон, и, компю...
Name: lemmas, dtype: object
